In [1]:
import keras

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [89]:
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Conv1D, Flatten, MaxPooling1D
from keras.layers import Activation, LeakyReLU
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
import pretty_midi

In [3]:
TIME_STEPS=10
MIN_TIME=0.1

In [4]:
def make_model():
    model=Sequential()
    model.add(LSTM(256, input_shape=(TIME_STEPS,88), return_sequences=True))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv1D(filters=256, kernel_size=5))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv1D(filters=128, kernel_size=2))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Flatten())
    model.add(Dense(88, activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer="adam")
    return model

In [7]:
model=make_model()
#model=load_model("model_20.hdf5")

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 258)           358104    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 10, 258)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 6, 258)            333078    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 6, 258)            0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 3, 258)            0         
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 3, 258)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 2, 128)            66176     
__________

In [52]:
midi_data = pretty_midi.PrettyMIDI('musics/fantaisie.mid')

In [53]:
midi_data.instruments

[Instrument(program=1, is_drum=False, name="Piano right"),
 Instrument(program=1, is_drum=False, name="Piano left")]

In [54]:
notes=[]

for j in range(1): #len(midi_data.instruments) for all instruments 
    temp=[]
    for i in range(len(midi_data.instruments[j].notes)):
        if(i>0):
            diff=midi_data.instruments[j].notes[i].start-midi_data.instruments[j].notes[i-1].start
            for k in range(int(diff/MIN_TIME)):
                temp.append(np.full((88), 0))
        pitch_format=np.full((88), 0)
        pitch_format[midi_data.instruments[j].notes[i].pitch-21]=1
        temp.append(pitch_format)
    notes.append(temp)

In [55]:
models=[]
for j in range(1): #len(midi_data.instruments) for all instruments 
    models.append(make_model())

In [56]:
def get_in_out(data, n):
    inp=[]
    out=[]
    for i in range(int(len(data))-(n-1)):
        pinp=[]
        for j in range(n):
            pinp.append(data[i+j])  
        inp.append(pinp)
        if(i<len(data)-n):
            out.append(data[i+n])
    return inp, out
data_in=[]
data_out=[]
for i in range(1): #len(midi_data.instruments) for all instruments 
    x, y=get_in_out(notes[i], TIME_STEPS)
    data_in.append(x)
    data_out.append(y)

In [57]:
Input_train=[]
Input_test=[]
Output_train=[]
Output_test=[]
for j in range(1): #len(midi_data.instruments) for all instruments 
    Input=np.array(data_in[j][:-1]).reshape(len(data_in[j][:-1]),TIME_STEPS, 88)
    Output=np.array(data_out[j]).reshape(len(data_out[j]), 88)
    Input_train.append(Input[:len(Input)//2])
    Input_test.append(Input[len(Input)//2:])
    Output_train.append(Output[:len(Output)//2])
    Output_test.append(Output[len(Output)//2:])

In [97]:
import math
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.9
    epochs_drop = 3.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [98]:
for j in range(1): #len(midi_data.instruments) for all instruments 
    checkpoint = ModelCheckpoint("models/model_"+str(j)+".hdf5", save_best_only=False)
    lrate = LearningRateScheduler(step_decay)
    models[j].fit(Input_train[j], Output_train[j], epochs=100, callbacks=[checkpoint, lrate])

Epoch 1/20
1627/1627 [==============================] - 64s 40ms/step - loss: 0.0606
Epoch 2/20
1627/1627 [==============================] - 23s 14ms/step - loss: 0.0597
Epoch 3/20
1627/1627 [==============================] - 25s 15ms/step - loss: 0.0598
Epoch 4/20
1627/1627 [==============================] - 26s 16ms/step - loss: 0.0589
Epoch 5/20
1627/1627 [==============================] - 24s 15ms/step - loss: 0.0589
Epoch 6/20
1627/1627 [==============================] - 25s 15ms/step - loss: 0.0586
Epoch 7/20
1627/1627 [==============================] - 25s 15ms/step - loss: 0.0586
Epoch 8/20
1627/1627 [==============================] - 25s 15ms/step - loss: 0.0584
Epoch 9/20
1627/1627 [==============================] - 26s 16ms/step - loss: 0.0584
Epoch 10/20
1627/1627 [==============================] - 25s 15ms/step - loss: 0.0583
Epoch 11/20
1627/1627 [==============================] - 26s 16ms/step - loss: 0.0583
Epoch 12/20
1627/1627 [==============================] - 27s 17

In [115]:
pred_notes = []
for j in range(1):  #len(midi_data.instruments) for all instruments   
    pred_notes.append(models[j].predict(np.array(Input_test[j]).reshape(len(Input_test[j]),TIME_STEPS, 88)))

In [123]:
import random

def last_N(data, N):
    last=[]
    for i in range(N):
        last.append(data[len(data)-N+i])
    return last
def gen_N(N):
    data=[]
    for i in range(N):
        p=np.full((1, 88), 0).reshape(88)
        p[random.randint(0,87)]=1
        data.append(p)
    return data
pred_notes = []
for j in range(1):  #len(midi_data.instruments) for all instruments 
    temp=gen_N(TIME_STEPS)
    for i in range(len(Input_test[j])):
        last=last_N(temp, TIME_STEPS)
        pred=models[j].predict(np.array(last).reshape(1, TIME_STEPS, 88))[0]
        temp.append(pred)
        if(i%50==0):
            temp.append(gen_N(1)[0]) #add a little randomness
    pred_notes.append(temp)
        #print(np.argmax(last[0]))

In [124]:
instrument_c_chord = pretty_midi.PrettyMIDI()

for j in range(1): #len(midi_data.instruments) for all instruments 
    instrument = pretty_midi.Instrument(program=midi_data.instruments[j].program, is_drum=midi_data.instruments[j].is_drum)
    time=0
    mult=1
    for i in range(len(pred_notes[j])):

        
        if(np.amax(pred_notes[j][i])!=0):
            mult=1.2
            time+=1.2*MIN_TIME
            endt=time+0.5
            _pitch=np.argmax(pred_notes[j][i])+21 
            
            note = pretty_midi.Note(velocity=127, pitch=min(int(_pitch),108), start=float(time), end=float(endt))
            
            instrument.notes.append(note)
        else:
            
            time+=mult*MIN_TIME
    
    instrument_c_chord.instruments.append(instrument)
# Write out the MIDI data
instrument_c_chord.write('compositions/new_random_fantaisie_2.mid')